In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
import socket
import env
import wrangle

In [6]:
df = wrangle.wrangle_logs()





1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [7]:
df.head()

,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,datetime,ip_name
date,,,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,2018-01-26 09:55:03,097-105-019-061.biz.spectrum.com
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,2018-01-26 09:56:02,097-105-019-061.biz.spectrum.com
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,2018-01-26 09:56:05,097-105-019-061.biz.spectrum.com
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,2018-01-26 09:56:06,097-105-019-061.biz.spectrum.com
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0,2018-01-26 09:56:24,097-105-019-061.biz.spectrum.com


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 to 2021-04-21
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        900223 non-null  object        
 1   path        900222 non-null  object        
 2   user_id     900223 non-null  int64         
 3   cohort_id   847330 non-null  float64       
 4   ip          900223 non-null  object        
 5   name        847330 non-null  object        
 6   slack       847330 non-null  object        
 7   start_date  847330 non-null  object        
 8   end_date    847330 non-null  object        
 9   created_at  847330 non-null  object        
 10  updated_at  847330 non-null  object        
 11  deleted_at  0 non-null       object        
 12  program_id  847330 non-null  float64       
 13  datetime    900223 non-null  datetime64[ns]
 14  ip_name     836679 non-null  object        
dtypes: datetime64[ns](1), float64(2), i

In [29]:
df.groupby('name')['path'].value_counts()

name       path                     
Andromeda  /                            1174
           toc                           638
           javascript-i                  516
           spring                        487
           java-iii                      433
                                        ... 
Zion       uploads/598dc43df39e2.jpg       1
           web-design/intro                1
           web-design/ui/visuals           1
           web-design/ux/layout            1
           web-design/ux/purpose           1
Name: path, Length: 13565, dtype: int64

In [102]:
df.groupby('program_id')['path'].value_counts()


program_id  path                                       
1.0         /                                              1681
            index.html                                     1011
            javascript-i                                    736
            html-css                                        542
            spring                                          501
                                                           ... 
3.0         where                                             1
4.0         content/html-css                                  2
            /                                                 1
            content/html-css/gitbook/images/favicon.ico       1
            content/html-css/introduction.html                1
Name: path, Length: 3309, dtype: int64

In [41]:
df_p01 = df[df['program_id'] == 1]
df_p02 = df[df['program_id'] == 2]
df_p03 = df[df['program_id'] == 3]

In [117]:
df_p03['path'].value_counts()

/                                           8358
search/search_index.json                    2203
classification/overview                     1785
1-fundamentals/modern-data-scientist.jpg    1655
1-fundamentals/AI-ML-DL-timeline.jpg        1651
                                            ... 
12-distributed-ml/4-acquire                    1
12-distributed-ml/5-inspect                    1
12-distributed-ml/6.1-prepare-part-1           1
imports                                        1
joins                                          1
Name: path, Length: 682, dtype: int64

In [67]:
counts_df = pd.DataFrame(df['path'].value_counts())
counts_df.reset_index(inplace = True)
counts_df.rename(columns = {'index':'lesson'}, inplace = True)

In [135]:
def top_lesson(dfa):
    '''
    This function generates top-accessed lesson from inputed dataframe
    and outputs name
    '''
    lessons = []
    dfa = pd.DataFrame(dfa['path'].value_counts())
    dfa.reset_index(inplace = True)
    dfa.rename(columns = {'index':'top_lesson'}, inplace = True)
    for x in dfa['top_lesson']:
            if x == '/':
                continue
            elif 'search' in x:
                continue
            elif 'index'  in x:
                continue
            else:
                lessons.append(x)
    lessons = pd.DataFrame(lessons)
    lessons.rename(columns = {0:'top lesson'}, inplace = True)
    return lessons.head(1)

In [136]:
def prog_split(dfa):
    '''
    This function splits the DataFrame up by program and outputs
    them as separate DataFrames
    '''
    df_p01 = df[df['program_id'] == 1]
    df_p02 = df[df['program_id'] == 2]
    df_p03 = df[df['program_id'] == 3]
    return df_p01, df_p02, df_p03

In [137]:
top_lesson(df) #calls top lesson function on overall DataFrame

,top lesson
0,javascript-i


In [138]:
#calls function on overall DataFrame to split into programs
df_p01, df_p02, df_p03 = prog_split(df) 
#call top lesson function on program 1
top_lesson(df_p01)

,top lesson
0,javascript-i


In [139]:
#call top lesson function on program 1
top_lesson(df_p02)

,top lesson
0,javascript-i


In [140]:
#call top lesson function on program 1
top_lesson(df_p03)

,top lesson
0,classification/overview
